# 오디오 멀티모달 Inference 코드

In [1]:
import warnings
warnings.filterwarnings('ignore')
import argparse
import numpy as np
import pandas as pd
import librosa
import joblib

In [4]:
def call_data(data_path):
    data, sr = librosa.load(data_path, sr=sample_rate)
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=n_mfcc)

    mean_mfcc = []
    mfccs = []
    for m in mfcc:
        mean_mfcc.append(np.mean(m))
    mfccs.append(mean_mfcc)
    
    meta_data = {'age':[age],
                 'gender':[gender],
                 'respiratory':[respiratory],
                 'fever_or_muscle':[fever_or_muscle]
                 }
    df_meta = pd.DataFrame(meta_data)
    df_mfcc = pd.DataFrame(mfccs, columns=['mfcc'+str(i+1) for i in range(n_mfcc)])
    df = df_meta.join(df_mfcc)
    df.to_csv('./data_preprocessing.csv', index=False)

    return df

In [13]:
def test_model(data):
    model = joblib.load(model_path)
    pred = model.predict(data)
    print(pred)

### 변수설정

In [15]:
# 환자정보
age = 30
gender = 0 # 0:male, 1:female
respiratory = 0 # 호흡기질환 0:없음, 1:있음
fever_or_muscle = 0 # 발열 또는 근육통 0:없음, 1:있음

# 오디오 변환 설정
n_mfcc = 30
sample_rate = 16000

### 오디오데이터와 환자정보 불러오기

In [7]:
data_path = './covid19/00001.wav'
data = call_data(data_path)
data

,age,gender,respiratory,fever_or_muscle,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,...,mfcc21,mfcc22,mfcc23,mfcc24,mfcc25,mfcc26,mfcc27,mfcc28,mfcc29,mfcc30
0,30,0,0,0,-274.876953,28.882778,-18.685369,-8.305733,-8.625875,-9.019613,...,0.019214,1.814102,-2.022824,1.866372,-0.656508,1.862009,0.04631,2.193763,-4.527728,1.044034


### 불러온 정보 모델에 넣어 covid19 예측하기

In [16]:
model_path = './SMOTE_RF.pkl'
test_model(data)

[0]
